In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns

train_data = pd.read_csv('/kaggle/input/banking-dataset-classification/new_train.csv')
df = pd.DataFrame(train_data)
df.head(5)

In [ ]:
df.info()

In [ ]:
nan_cols = [i for i in df.columns if df[i].isnull().any()]
df[nan_cols].isnull().sum()

In [ ]:
dftNumeric = df.select_dtypes(include=['int64', 'float64'])
dftObject = df.select_dtypes(include=['object'])
dftNumeric.info()

In [ ]:
dftObject.info()

In [ ]:
dftObjcat = dftObject.columns
for i in dftObjcat:
    print(f'Catagory of {i}')
    catlist = dftObject[i].unique()
    for j, val in enumerate(catlist):
         dftobjfinal = dftObject[i].replace({val:j+1},inplace=True)
         #print(dftobjfinal)
         print(j,val)
    print('--------------------------------')

In [ ]:
dftObject

In [ ]:
df_Final = pd.concat([dftObject, dftNumeric], axis=1)
df_Final

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor()
y_pred = clf.fit_predict(df_Final)

In [ ]:
x_score = clf.negative_outlier_factor_
outlier_score = pd.DataFrame()
outlier_score["score"] = x_score

#threshold
threshold2 = -1.5
filtre2 = outlier_score["score"] < threshold2
outlier_index = outlier_score[filtre2].index.tolist()

In [ ]:
len(outlier_index)

In [ ]:
df_Final.drop(outlier_index, inplace=True)

In [ ]:
f,ax = plt.subplots(figsize=(12,8))
sns.heatmap(df_Final.corr(), cmap="PuBu", annot=True, linewidths=0.5, fmt= '.2f',ax=ax)
plt.show()

In [ ]:
corr = df_Final.corr()
corr["previous"].sort_values()

In [ ]:
dfmodel = df_Final.drop(['campaign','duration'], axis = 1)

In [ ]:
x = dfmodel.drop(['y'], axis=1)
x

In [ ]:
y = dfmodel['y']
y

In [ ]:
y.value_counts()

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x=y)
plt.title('Count of Unique Values in y')
plt.xlabel('y')
plt.ylabel('Count')
plt.show()

In [ ]:
from sklearn import preprocessing
x= preprocessing.StandardScaler().fit(x).transform(x)
x[0:5]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Validation set:', X_test.shape,  y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import warnings
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from PIL import Image
warnings.filterwarnings('ignore')

In [ ]:
classifiers = [
    LogisticRegression(solver='liblinear'),
    RandomForestClassifier(),
    KNeighborsClassifier(),           
    SVC(),
    MLPClassifier(),
    GaussianNB(),
    DecisionTreeClassifier(max_depth=8, random_state=0, splitter='random')
]

In [ ]:
tacc, vacc = [], []
for i in classifiers:
    model = i
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('______________________________________________')
    print(i)
    print('..............................................')
    tacc.append(model.score(X_train, y_train)*100)
    vacc.append(model.score(X_test, y_test)*100)
    print('Train',tacc[-1])
    print('Test',vacc[-1])

In [ ]:
models = pd.DataFrame({'Model': ['LogisticRegression', 'RandomForestClassifier', 'KNeighborsClassifier', 'Linear SVC', 'Multi-Layer Perceptron', 'Gaussian Naive Bayes', 'Decision Tree'],
    'Train Score': tacc,
    'Test Score': vacc})
models.sort_values(by='Test Score', ascending=False)

In [ ]:
#Data Preprocessing for transformer
dftfNumeric = df.select_dtypes(include=['int64', 'float64'])
dftfCat = df.select_dtypes(include=['object'])
dftfNumeric.info()
dftfCat.info()

In [ ]:
dftf_Final = pd.concat([dftfCat, dftfNumeric], axis=1)

In [ ]:
dftfmodel = dftf_Final.drop(['campaign','duration'], axis = 1)

In [ ]:
# FINAL Transformer and Ensembe Learner

#-----------------------------------------------------------------

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

# Assuming 'dftfmodel' is your preprocessed DataFrame
# Update categorical columns based on your dataset
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

# Encode categorical features
label_encoders = {}
for column in categorical_columns:
    label_encoders[column] = LabelEncoder()
    dftfmodel[column] = label_encoders[column].fit_transform(dftfmodel[column])

# Split dataset into features and target
y = dftfmodel['y']
X = dftfmodel.drop(['y'], axis=1)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
def convert_to_binary(label):
    return 1 if label.lower() == 'yes' else 0

y_train = y_train.apply(convert_to_binary)
y_test = y_test.apply(convert_to_binary)


# Define vocabulary size for categorical features
vocab_size = max(X_train.max()) + 1

# Function to build a basic Transformer model
def build_transformer_model(max_length, vocab_size, embedding_dim=64, num_heads=8, dense_dim=32, dropout_rate=0.1):
    inputs = Input(shape=(max_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(inputs)
    attention_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(embedding_layer, embedding_layer)
    attention_output = Dropout(dropout_rate)(attention_output)
    pooled_output = GlobalAveragePooling1D()(attention_output)
    dense_layer = Dense(dense_dim, activation='relu')(pooled_output)
    outputs = Dense(1, activation='sigmoid')(dense_layer)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Pad sequences for categorical features (assuming X_train and X_test are numpy arrays)
max_length = X_train.shape[1]  # Maximum sequence length (number of categorical features)
X_train_padded = pad_sequences(X_train.values, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test.values, maxlen=max_length, padding='post')

# Build and compile the Transformer model
transformer_model = build_transformer_model(max_length, vocab_size)
transformer_model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

# Train the Transformer model
transformer_model.fit(X_train_padded, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Generate predictions from the Transformer model for the test set
transformer_outputs = transformer_model.predict(X_test_padded)

# Train Random Forest and Gradient Boosting ensemble models
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

# Generate predictions from ensemble models
rf_preds = rf_model.predict_proba(X_test)[:, 1]  # Use class probabilities for binary classification
gb_preds = gb_model.predict_proba(X_test)[:, 1]

# Concatenate ensemble predictions and Transformer outputs for MetaModel1
concatenated_features_meta1 = np.column_stack((rf_preds, gb_preds))

# Define the architecture for MetaModel1
input_layer_meta1 = Input(shape=(concatenated_features_meta1.shape[1],))
dense_layer_meta1 = Dense(64, activation='relu')(input_layer_meta1)
dropout_layer_meta1 = Dropout(0.3)(dense_layer_meta1)
output_layer_meta1 = Dense(1, activation='sigmoid')(dropout_layer_meta1)

# Create MetaModel1
meta_model1 = Model(inputs=input_layer_meta1, outputs=output_layer_meta1)
meta_model1.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

# Train MetaModel1
meta_model1.fit(concatenated_features_meta1, y_test, epochs=50, batch_size=32, validation_split=0.2)

# Generate predictions from MetaModel1
meta_model1_preds = meta_model1.predict(concatenated_features_meta1)

# Concatenate features for MetaModel2
concatenated_features_meta2 = np.column_stack((transformer_outputs, gb_preds))

# Define the architecture for MetaModel2
input_layer_meta2 = Input(shape=(concatenated_features_meta2.shape[1],))
dense_layer_meta2 = Dense(64, activation='relu')(input_layer_meta2)
dropout_layer_meta2 = Dropout(0.3)(dense_layer_meta2)
output_layer_meta2 = Dense(1, activation='sigmoid')(dropout_layer_meta2)

# Create MetaModel2
meta_model2 = Model(inputs=input_layer_meta2, outputs=output_layer_meta2)
meta_model2.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

# Train MetaModel2
meta_model2.fit(concatenated_features_meta2, y_test, epochs=50, batch_size=16, validation_split=0.2)

# Generate predictions from MetaModel2
meta_model2_preds = meta_model2.predict(concatenated_features_meta2)

# Concatenate predictions from MetaModel1 and MetaModel2
concatenated_meta_preds = np.column_stack((meta_model1_preds, meta_model2_preds))

# Define the architecture for the combined model
input_layer_combined = Input(shape=(concatenated_meta_preds.shape[1],))
dense_layer_combined = Dense(64, activation='relu')(input_layer_combined)
dropout_layer_combined = Dropout(0.3)(dense_layer_combined)
output_layer_combined = Dense(1, activation='sigmoid')(dropout_layer_combined)

# Create the combined model
combined_model = Model(inputs=input_layer_combined, outputs=output_layer_combined)
combined_model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

# Train the combined model
combined_model.fit(concatenated_meta_preds, y_test, epochs=50, batch_size=16, validation_split=0.2)

In [ ]:
from sklearn.metrics import roc_curve, auc, confusion_matrix, f1_score

# Compute ROC curve for the combined model
y_pred_meta = combined_model.predict(concatenated_meta_preds)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_meta)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


# Predictions from meta-learner model
y_pred_meta_binary = (y_pred_meta > 0.5).astype(int)

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred_meta_binary)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Calculate F1 Score
f1 = f1_score(y_test, y_pred_meta_binary)
print("F1 Score:", f1)